In [1]:
from flask import Flask, request, jsonify
import base64
import tempfile
import os
from bob.io.audio import reader

In [ ]:
def decode_base64(base64string):
    b64_byte = base64.b64decode(base64string)
    wav_temp = tempfile.TemporaryFile()
    
    with open("./" + str(wav_temp.name) ,'wb') as wav: 
        wav.write(b64_byte)
    
    data = bob.io.audio.reader(str(wav_temp.name))
    
    if data.rate!=16000:
        return {'status':'fail',
                'msg':'acceptable sampling rate is 16kHz',
                'wav_temp' :wav_temp, 
                'data' : data}

    elif data.load().shape[0] != 1:
        return {'status' : 'fail',
                'msg' : 'mono channel audio required',
                'wav_temp' : wav_temp,
                'data' : data }
    

    return {'status': 'pass',
            'msg' : 'acceptable audio',
            'wav_temp' : wav_temp,
            'data' : data }

In [ ]:
def verifySpeaker(base64string):
    task      = request.get_json()['task']
    aud_b64   = request.get_json()['audio']
    spkr_id   = request.get_json()['speakerId']
    
    try:
        image = get_image(base64string)
    except:
        return jsonify(error="improper base64 encoding", message="failure!")
    
    
    
    